In [2]:
from pprint import pprint
import time
from transformers import pipeline
from src.tcp_llm import listen_for_prompt, StandbyLLM
import asyncio

In [3]:

model_path = 'openlm-research/open_llama_7b'

def get_llm(model_path: str) -> pipeline:
    """
    Get the LLama model.

    Args:
        model_path (str): The path to the model.

    Returns:
        pipeline: The LLama model.
    """
    llm = pipeline(
        'text-generation',
        model=model_path,
        tokenizer=model_path,
        device=0,
        framework='pt',
    )
    # Test the llm
    test_prompt = 'Navy-seals have this saying before going into battle: "Go get '
    res_test = llm(test_prompt, max_length=20, do_sample=True, top_k=10, num_return_sequences=1)
    pprint(res_test)

    return llm


In [4]:

port = 9999
log_msg = "\n[LOG] - {time} - {msg} \n"
llm = get_llm(model_path)
llama = StandbyLLM(llm)
pprint(log_msg.format(time=time.time(), msg="LLama loaded 🦙"))
prompt_gen = listen_for_prompt(port) 


Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.22s/it]
You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'generated_text': 'Navy-seals have this saying before going into battle: "Go '
                    'get 2. Go get'}]
'\n[LOG] - 1690743562.817288 - LLama loaded 🦙 \n'


In [7]:
from src.tcp_llm import listen_for_prompt, StandbyLLM

first_prompt = await prompt_gen.__anext__()
pprint(first_prompt)

'Llama\n'


In [8]:

while True:
    prompt = await prompt_gen.__anext__()
    # prompt = input("Enter a prompt: ")
    pprint(log_msg.format(time=time.time(), msg=prompt))
    for llama_res in llama.run_convo(prompt):
        pprint(llama_res)
        res = llm(llama_res)
        pprint(res)

        

Stop program 
 {{#user~}} 
You are a helpful AI code-writing assistant, the perfect data analyst who is jovial, fun and writes great code to solve data problems!

Answer my questions with both text describing your plan (but not an answer), and then the code in markdown that will be executed!

* Use `print` to show results.
* Don't answer the question directly, instead suggest how you will solve the problem, then write in a ```python markdown block, the code you will use to solve the problem.
* For plotting, please use `matplotlib`. use `plt.show()` to display the plot to the user.
 {{~/user}} 
 {{#each conversation}} 
 {{#if (equal this.role 'user')}} 
 {{#user~}} 
 {{this.content}} 
 {{~/user}} 
 {{/if}} 
 {{#if (equal this.role 'assistant')}} 
 {{#assistant~}} 
 {{this.content}} 
 {{~/assistant}} 
 {{/if}} 
 {{/each}} 

 {{#assistant~}} 
 {{gen "thoughts" temperature=0.1 max_tokens=120 stop=["```", "<|end|>"]}} 
```python
 {{gen "code" temperature=0.0 max_tokens=800 stop=["```", "<|end|>"]}} 
 {{~/assistant}}

'\n```python\n\n```'
